In [1]:
import geopandas as gpd
import os
import pandas as pd
import glob
import datetime

In [2]:
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
fold = r"C:\Users\arich\Desktop\Shape Files GIS-20240311T200548Z-001\Shape Files GIS"
main =  os.walk(fold)

sites = {}
for path, dirnames, filenames in main:
    
    for file in filenames:
        if  file.endswith('.shp') and ("sites" in file.lower()):
            print(file)
            # print('{} {} {}'.format(repr(path), repr(dirnames), repr(file)))
            cur = gpd.read_file(os.path.join(path, file))
            sites[file] = cur
            # df.extend([cur])

201607_SR_Sites_with_Analysis.shp
201609_SR_Sites_with_Analysis.shp
201611_SR_Sites_with_Analysis.shp
201701_SR_Sites_with_analysis.shp
201703_SR_Sites_with_analysis_2.shp
201705_SR_Sites_with_analysis.shp
201707_SR_Sites_with_Analysis.shp
201709_SR_Sites_with_analysis.shp
201711_SR_Sites_with_analysis.shp
201711_SR_Sites_with_analysis.shp
201801_SR_Sites_with_analysis.shp
201803_SR_Sites_with_analysis.shp
201805_SR_Sites_with_analysis.shp
201807_SR_Sites_with_Analysis.shp
201810_SR_Sites_with_Analysis.shp
201901_SR_Sites_with_Analysis.shp
Seepage_Run_Sites_July2021.shp
Seepage_Run_Sites_March2021.shp
Seepage_Run_Sites_May2021.shp
Seepage_Run_Sites_November2021.shp
Seepage_Run_Sites_September2021.shp
Seepage_Run_Sites_January2022.shp
Seepage_Run_Sites_July2022.shp
Seepage_Run_Sites_May2022.shp
Seepage_Run_Sites_November2022.shp
Seepage_Run_Sites_September2022.shp
Seepage_Run_Sites_November2023.shp
Seepage_Run_Sites_January2024.shp


In [7]:
import conda_scripts
import numpy as np

In [85]:
# Path to the output Excel file
output_file = 'discharge\meas.xlsx'

# Create an Excel writer object
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name, gdf in sites.items():
        # Convert GeoDataFrame to DataFrame
        df = pd.DataFrame(gdf.drop(columns='geometry'))
        # Write DataFrame to Excel sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

<>:2: DeprecationWarning: invalid escape sequence \m
<>:2: DeprecationWarning: invalid escape sequence \m
C:\Users\arich\AppData\Local\Temp\1\ipykernel_23440\3155978207.py:2: DeprecationWarning: invalid escape sequence \m
  output_file = 'discharge\meas.xlsx'
C:\Users\arich\AppData\Local\mambaforge\envs\gis_py_10\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\arich\AppData\Local\mambaforge\envs\gis_py_10\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\arich\AppData\Local\mambaforge\envs\gis_py_10\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is mo

In [121]:
alls = []
for k in sites.keys():
    cols = 'WPName 	 	Date 	 	LAT 	LON  Discharge	SiteName CreekCode geometry  source'.split()
    dfo = sites[k]
    # dfo.loc[:,'source'] = k
    if 'Discharge' in dfo.columns:
        dfo = dfo.astype({'Discharge':float})
        # print(dfo.head(),'\n\n', '-'*20,'\n')
        cols = [x for x in cols if x in dfo.columns]
        alls.extend([dfo.loc[:,cols]])
    else:
        col = dfo.drop(columns = ['geometry','source']).columns[-1]
        # cols = 'WPName 	SiteName 	Date 	 	LAT 	LON 	CreekCode geometry'.split()
        cols = [x for x in cols if x in dfo.columns]
        cols.extend([col])
        
        alls.extend([dfo.loc[:,cols].rename(columns = {col:'Discharge'})])


    
alls = pd.concat([a.to_crs(2226) for a in alls])
alls = alls.loc[alls.geometry.x<np.inf]

In [122]:
alls.to_file('discharge\streamQ.shp')

<>:1: DeprecationWarning: invalid escape sequence \s
<>:1: DeprecationWarning: invalid escape sequence \s
C:\Users\arich\AppData\Local\Temp\1\ipykernel_23440\3008717688.py:1: DeprecationWarning: invalid escape sequence \s
  alls.to_file('discharge\streamQ.shp')
